In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.backends.backend_pdf import PdfPages
import os

# Set location for pdfs to be saved
# Change to your system
date = "07-31-24"
team = "Wareham"
output_directory = f'/Users/seanmcgarry/Desktop/Wareham/Scouting/ScoutReports/{team}/Hitters/SprayCharts/{date}'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
os.chdir(output_directory)
# Load the Trackman file
df = pd.read_csv('/Users/seanmcgarry/Desktop/Wareham/Data/all_games.csv')

# Data cleaning and transformation
df = df.dropna(subset=['Distance', 'Bearing'])
df['swing'] = np.where(df['PitchCall'].isin(['InPlay', 'StrikeSwinging', 'FoulBall']), 1, 0)
df['whiff'] = np.where(df['PitchCall'] == 'StrikeSwinging', 1, 0)
df['Count'] = df['Balls'].astype(str) + ' - ' + df['Strikes'].astype(str)

# Calculate hc_x and hc_y
df['hc_x'] = np.sin(df['Bearing'] * np.pi / 180) * df['Distance']
df['hc_y'] = np.cos(df['Bearing'] * np.pi / 180) * df['Distance']

# Select all playable batted balls
data = df[abs(df['Bearing']) <= 55]

# Filter the dataset for desired team using abbreviation
team = data[data['BatterTeam'] == 'WAR_GAT']

# Or Input a list of batter names
#batter_names = ["Hot, Bayram", "Summerhill, Brendan", "Gray, Triston", "Putnam, Eli", "Jimenez, Antonio", "Dessureault, Yohann",
     #      "Earley, Nate", "Turner, Colby", "Jarrell, Jacob", "White, Sam", "Garcia, Aris", "Chapman, Reese", "Lippe, Michael"] 
#team = data[data['Batter'].isin(batter_names)]

# Get unique hitters
hitters = team['Batter'].unique()

# Function to plot the baseball field
# Approximated based on CCBL Fields
def plot_baseball_field(ax):
    # Baseball field coordinates (simplified)
    home_plate = [0, 0]
    first_base = [90, 90]
    second_base = [0, 180]
    third_base = [-90, 90]
    
    # Draw bases
    bases = [home_plate, first_base, second_base, third_base, home_plate]
    bases = np.array(bases)
    ax.plot(bases[:, 0], bases[:, 1], color='black')
    
    # Draw the foul lines
    fence_radius = 345
    ax.plot([0, -fence_radius * np.cos(np.pi / 4)], [0, fence_radius * np.sin(np.pi / 4)], color='black')
    ax.plot([0, fence_radius * np.cos(np.pi / 4)], [0, fence_radius * np.sin(np.pi / 4)], color='black')
    
    # Draw the outfield fence as a smooth arc
    angles = np.linspace(-45, 45, 500)  # More points for a smoother arc
    angles_rad = np.radians(angles)
    
    # Quadratic Bezier curve for smooth transition between distances
    control_point = (0, 380)
    start_point = (-315 * np.sqrt(2) / 2, 315 * np.sqrt(2) / 2)
    end_point = (315 * np.sqrt(2) / 2, 315 * np.sqrt(2) / 2)
    distances = 315 + (380 - 315) * (1 + np.cos(np.radians(angles * 2))) / 2
    
    x = distances * np.sin(angles_rad)
    y = distances * np.cos(angles_rad)
    ax.plot(x, y, color='black')
    
    ax.set_aspect('equal')
    ax.axis('off')

# Create a custom colormap
colors = [(0, 0, 1), (0, 0, 0), (1, 0, 0)]  # Blue, Black, Red
n_bins = 100  # Discretize into 100 bins
cmap_name = 'custom_cmap'
custom_cmap = mcolors.LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)

# Generate a spray chart for each hitter
for hitter in hitters:
    hitter_data = team[team['Batter'] == hitter]
    
    plt.figure(figsize=(10, 10))
    sc = plt.scatter(hitter_data['hc_x'], hitter_data['hc_y'], c=hitter_data['ExitSpeed'], cmap=custom_cmap, s=50)  # s=50 for bigger dots
    cb = plt.colorbar(sc, label='EV (mph)')
    
    ax = plt.gca()
    plot_baseball_field(ax)
    
    plt.title(f'Spray Chart for {hitter} as of {date}')
    # Save the plot as a PDF 
    pdf_filename = f'{hitter}_spray_chart.pdf'
    plt.savefig(pdf_filename, format='pdf')
    plt.close()

